In [1]:
import hd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import importlib

from tensorflow.python.client import timeline

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
sess = tf.Session(config=config)
cpu_session = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))
run_metadata = tf.RunMetadata()

In [13]:
vectors = hd.vectors.space_graph_altered_permutations([5, 5, 3, 2, 1, 1], bounds=(-1.0, 1.0))

In [14]:
root_valences = hd.tenney.hd_root_valence(vectors)
downhill = tf.boolean_mask(vectors, root_valences <= 0.0)
hds = hd.tenney.hd_aggregate_graph(downhill[:, None, :])
reasonable = tf.boolean_mask(downhill, hds < 8.0)
reasonable = tf.stack([tf.zeros_like(reasonable), reasonable], 1)

In [22]:
hds = hd.tenney.hd_aggregate_graph(reasonable) + 1.0
pds = hd.tenney.pd_aggregate_graph(reasonable)
hds_np, pds_np = sess.run([hds, pds])
hds = tf.constant(hds_np)
pds = tf.constant(pds_np)

In [23]:
N_POINTS = 1024
C = 0.02
LEARNING_RATE = 1.0e-3
CONVERGENCE_THRESHOLD = 1.0e-12
MAX_ITERS = 1000000

xs = np.linspace(-1.0, 1.0, N_POINTS)

In [24]:
with tf.variable_scope("default", reuse=tf.AUTO_REUSE):
    log_pitches = tf.get_variable("log_pitches_1024x2", [N_POINTS, 2], dtype=tf.float64)
    init_op = tf.global_variables_initializer()

In [25]:
loss = hd.optimize.parabolic_loss_function(pds, hds, log_pitches, a=C, b=C)
stopping_op = hd.optimize.stopping_op(loss, [log_pitches], lr=LEARNING_RATE, ct=CONVERGENCE_THRESHOLD)

In [26]:
xys = np.stack([np.zeros_like(xs), xs], 1)

In [27]:
sess.run(init_op)
sess.run(log_pitches.assign(xys))
sess.run(loss)

array([51.        , 50.80468769, 50.60975759, ...,  3.0022933 ,
        3.00057332,  3.        ])

In [ ]:
for idx in range(MAX_ITERS):
    if (sess.run(stopping_op, run_metadata=run_metadata)):
        print("Converged at iteration: ", idx)
        out_pitches = np.array(sess.run(log_pitches))
        print(out_pitches)
        break

In [37]:
log_vectors = hd.vectors.pd_graph(reasonable)
diffs_to_poles = tf.abs(tf.tile(log_vectors[:, None, None], [1, 1, 1]) - out_pitches)
mins = tf.argmin(diffs_to_poles, axis=0)
winner = tf.map_fn(lambda m: tf.map_fn(lambda v: reasonable[v], m, dtype=tf.float64), mins, dtype=tf.float64)
winners = sess.run(winner)
print(winners)

def vector_to_ratio(vector):
    primes = hd.PRIMES[:vector.shape[0]]
    num = np.where(vector > 0, vector, np.zeros_like(primes))
    den = np.where(vector < 0, vector, np.zeros_like(primes))
    return (
        np.product(np.power(primes, num)), 
        np.product(primes ** np.abs(den))
    )

all_possible_pitches = set()

for row in winners:
    all_possible_pitches.add(tuple([vector_to_ratio(r) for r in row]))

print(len(all_possible_pitches))
for pit in sorted(all_possible_pitches, key=lambda r: (r[0][0] / r[0][1])):
    print(pit[0])

[[[-1.  0.  0.  0.  0.  0.]]

 [[-1.  0.  0.  0.  0.  0.]]

 [[-1.  0.  0.  0.  0.  0.]]

 ...

 [[ 1.  0.  0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.  0.  0.]]]
46
(1.0, 2.0)
(11.0, 21.0)
(8.0, 15.0)
(7.0, 13.0)
(6.0, 11.0)
(5.0, 9.0)
(4.0, 7.0)
(3.0, 5.0)
(8.0, 13.0)
(7.0, 11.0)
(2.0, 3.0)
(9.0, 13.0)
(5.0, 7.0)
(8.0, 11.0)
(11.0, 15.0)
(10.0, 13.0)
(7.0, 9.0)
(4.0, 5.0)
(9.0, 11.0)
(11.0, 13.0)
(6.0, 7.0)
(13.0, 15.0)
(8.0, 9.0)
(10.0, 11.0)
(12.0, 13.0)
(14.0, 15.0)
(1.0, 1.0)
(16.0, 15.0)
(14.0, 13.0)
(12.0, 11.0)
(10.0, 9.0)
(8.0, 7.0)
(6.0, 5.0)
(16.0, 13.0)
(14.0, 11.0)
(4.0, 3.0)
(18.0, 13.0)
(10.0, 7.0)
(16.0, 11.0)
(14.0, 9.0)
(8.0, 5.0)
(18.0, 11.0)
(12.0, 7.0)
(16.0, 9.0)
(20.0, 11.0)
(2.0, 1.0)
